## Manual database creation

In [1]:
import os
import pandas as pd
import numpy as np
import brightway2 as bw

In [2]:
bw.projects.set_current("bw_idaes")
bw.databases

Databases dictionary with 3 object(s):
	IDAES
	biosphere3
	ecoinvent 3.7.1_apos_ecoSpold02

In [3]:
eidb = 'ecoinvent 3.7.1_apos_ecoSpold02'
bio = 'biosphere3'

In [4]:
# Search for the elementary flow of interest

search_name = 'Carbon dioxide'  # Mind capital letters

for flow in bw.Database("biosphere3"):
    if search_name in flow['name'] :
        print(flow, flow.key)

'Carbon dioxide, from soil or biomass stock' (kilogram, None, ('air', 'non-urban air or from high stacks')) ('biosphere3', '78eb1859-abd9-44c6-9ce3-f3b5b33d619c')
'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'low population density, long-term')) ('biosphere3', '28e1e2d6-97ad-4dfd-932a-9edad36dcab9')
'Carbon dioxide, fossil' (kilogram, None, ('air', 'lower stratosphere + upper troposphere')) ('biosphere3', '16eeda8a-1ea2-408e-ab37-2648495058dd')
'Carbon dioxide, in air' (kilogram, None, ('natural resource', 'in air')) ('biosphere3', 'cc6a1abb-b123-4ca6-8f16-38209df609be')
'Carbon dioxide, to soil or biomass stock' (kilogram, None, ('soil',)) ('biosphere3', '375bc95e-6596-4aa1-9716-80ff51b9da77')
'Carbon dioxide, from soil or biomass stock' (kilogram, None, ('air', 'urban air close to ground')) ('biosphere3', 'e8787b5e-d927-446d-81a9-f56977bbfeb4')
'Carbon dioxide, non-fossil, from calcination' (kilogram, None, ('air',)) ('biosphere3', '6900ccc1-4b7a-422a-9fad-7c31eb1fc46f')
'Ca

In [5]:
# Designate the elementary flow of interest
CO2 = ('biosphere3', '349b29d1-3e58-4c66-98b9-9d1a076efd2e')

# If you have any more
#CH4 = ('biosphere3', '0795345f-c7ae-410c-ad25-1845784c75f5')

In [6]:
# Create new database for IDAES integration
db1 = bw.Database('IDAES')
db1.register()

In [7]:
# Add an activity (Recommended: only one flow)
# Alternative 1
db1.write({('IDAES', 'Flash'):{
    'name': 'Flash',
    'exchanges': [
        {'amount': 0.04401,  # 0.04401 kg-CO2 / 1mol CO2
        'input':CO2,
        'type': 'biosphere'
        }
    ],
    'unit': 'kg',
    'location': 'CH',
    'categories': ('separation', 'idaes', 'unit')
}})

Writing activities to SQLite3 database:
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00
Title: Writing activities to SQLite3 database:
  Started: 05/07/2021 13:04:06
  Finished: 05/07/2021 13:04:06
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 2.17


In [13]:
# Check databases
bw.databases

Databases dictionary with 3 object(s):
	IDAES
	biosphere3
	ecoinvent 3.7.1_apos_ecoSpold02

In [14]:
idaes = bw.Database('IDAES')

In [15]:
idaes.random()

'Flash' (kg, CH, ('separation', 'idaes', 'unit'))

In [16]:
# Perform LCA
process = idaes.random()
CC_method = [m for m in bw.methods if 'IPCC 2013' in str(m) and  'climate change' in str(m) and 'GWP 100a' in str(m) and not 'no LT' in str(m)][0]

lca = bw.LCA({process:1}, CC_method)
lca.lci()
lca.lcia()

In [17]:
lca.score

0.04400999844074249

In [12]:
# To delete database created 
del bw.databases['database_name_here']